<a href="https://colab.research.google.com/github/ucfilho/Codigos_Teste/blob/master/ABCOptim_jun_15_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
'''Rosembrock Function'''
def Fun(x):
    fun=0
    a=1.0
    b=100.0
    fun = (a-x[0])**2 + b*(x[1]-x[0]**2)**2
    return fun

In [0]:
''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''
def FOBJ(X):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

In [0]:

'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:

'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x,Fun):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x,Fun)
    fit_ref=max(fit)
    
    for i in range(rows):
        prob[i] = 0.9*fit[i]/fit_ref+0.1

    return prob


In [0]:
''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X,Fun):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

In [0]:
'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest):
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST

In [0]:
'''
employed bee phase
'''
def EmployedBee(xo,x,MAX,MIN,trial) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):
            fi=np.random.uniform(low=-1.0, high=1.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
            if (V[i,j]>MAX[j]):
              V[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
            if (V[i,j]<MIN[j]):
              V[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
              
    YCAL=FOBJ(xo)
    YV=FOBJ(V)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
          trial[i]=0
          xo[i,]=np.copy(V[i,]) #MODIFICADO JUN 12
        else:
          trial[i]=trial[i]+1
            

    return xo,trial  #MODIFICADO JUN 12

In [0]:
'''
onlooker bee phase
'''
def OnlookerBee(xo,x,trial,MAX,MIN): #OnlookerBee(xo,x_Employed)
    rows = xo.shape[0]
    cols = xo.shape[1]
    prob=PROB(x,Fun)
    prob_ref=max(prob)
    #best=np.argmax(prob)
    
    for i in range(rows):
      prob_ref=np.random.uniform(low=0, high=1.0, size=None)
      rd=randint(0, (rows-1))
      if(prob[i] > prob_ref):  
        fi=np.random.uniform(low=-1.0, high=1.0, size=None)
        Xmi=xo[i,]+fi*(xo[i,]-x[rd,])
        xo[i,]=np.copy(Xmi)
        for j in range(cols):
          if (xo[i,j]>MAX[j]):
            xo[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
          if (xo[i,j]<MIN[j]):
            xo[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
    
    YCAL=FOBJ(xo)
    YV=FOBJ(x)
        
    for i in range(rows):
      if(YV[i]<YCAL[i]):
        trial[i]=0
        xo[i,]=np.copy(x[i,])
      else:
        trial[i]=trial[i]+1
    return xo,trial

In [0]:
'''
scout bee phase
'''
def ScoutBee(trial,x,ntrail,MAX,MIN):
#def ScoutBee(x,trial,MAX,MIN):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    for i in range(PAR):
      if(trial[i]>= ntrail):
        trial[i]=0
        for j in range(NPAR):
          x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x,trial

In [0]:
'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search
'''
def ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN):

    trial=np.zeros(NPAR) # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
    xbest=GetBest(xo,xbest)
    
    for i in range(ITE):
      x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para employed bee
      xo,trial=EmployedBee(xo,x,MAX,MIN,trial)
      xbest=GetBest(xo,xbest) #MODIFICADO JUN 12
      x=Enxame(PAR,NPAR,MAX,MIN)  # comparativo para OnlookerBee
      xo,trial=OnlookerBee(xo,x,trial,MAX,MIN)
      xbest=GetBest(xo,xbest) 
      #xbest=GetBest(xo,xbest)
      # x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para ScoutBee
      xo,trial=ScoutBee(trial,x,ntrail,MAX,MIN) #MODIFICADO JUN 12
      xbest=GetBest(xo,xbest) #MODIFICADO JUN 12
    x=np.copy(xo)
    xbest=GetBest(xo,xbest) #MODIFICADO JUN 12
    
    return xbest, x,trial



In [12]:
dim=2
MAX=np.repeat(5,dim)
MIN=np.repeat(-5,dim)

NPAR=175 #PARTICULAS (fontes de comida)
ITE=300 #ITERACOES
PAR=len(MAX)
ntrail=25 #numero de buscas ate abandonar uma fonte de alimento

for i in range(20):
    print("resolucao",i+1," ")
    xbest, x,trial =ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN)
    print("vetor",xbest,"funcao", Fun(xbest),"\n")



resolucao 1  
vetor [0.99183473 0.98507651] funcao 0.00024633146502497694 

resolucao 2  
vetor [1.01803068 1.04199965] funcao 0.003475896219977801 

resolucao 3  
vetor [1.01524958 1.02787561] funcao 0.0010482861275181005 

resolucao 4  
vetor [1.01597457 1.02854133] funcao 0.0015969373493208469 

resolucao 5  
vetor [1.0159272  1.03436402] funcao 0.000762605613822613 

resolucao 6  
vetor [0.99894047 0.9980147 ] funcao 2.88181877356559e-06 

resolucao 7  
vetor [1.05564811 1.1071785 ] funcao 0.008301496201446844 

resolucao 8  
vetor [1.13067589 1.27306964] funcao 0.01994735209321784 

resolucao 9  
vetor [1.05013509 1.0978518 ] funcao 0.004945892514562067 

resolucao 10  
vetor [0.96096665 0.92277437] funcao 0.0015701865754340224 

resolucao 11  
vetor [0.95429771 0.91159829] funcao 0.002172267172528721 

resolucao 12  
vetor [0.99600803 0.99229814] funcao 2.3019082056961125e-05 

resolucao 13  
vetor [0.98961819 0.9845184 ] funcao 0.00278505273135733 

resolucao 14  
vetor [0.99396